# Choice Paper

Bijesh Mishra, Ph.D.

Techno-economic analysis of agrivoltaic systems in Alabama. A paper for
<a href="https://www.aaea.org/publications/choices-magazine"
class="uri">Choice Magazine</a> by AAEA.

# 1. Setting Up

## 1.1 Housekeeping

In [ ]:
# #| echo: TRUE
rm(list = ls()) # Clean the environment.
options(
  warn=0, # Warnings. options(warn=-1) / options(warn=0)
  scipen=999 # No scientific notations.
  )

: 

## 1.2 Working directory

Codes and output are suppressed. Errors and warnings are visible. No
warning and no error means code is working as it should.

## 1.3 Load libraries

In [ ]:
library(tidyverse, warn.conflicts = FALSE, quietly = TRUE)

: 

In [ ]:
pacman::p_loaded()

: 

## 1.4 Progress Bar

Tracking data processing progress.

In [ ]:
####### Progress Bar #####
pb = progress_bar$new(
  format = "Processing data at :rate. Processed :bytes in :elapsed.",
  clear = TRUE,
  total = NA, 
  width = 80)
f = function() {
  for (i in 1:100) {
    pb$tick(sample(1:100 * 1000, 1))
    Sys.sleep(2/100)
  }
  pb$tick(1e7)
  #invisible()
}

: 

## 1.5 Theme for plots

Setting theme for plots:

In [ ]:
####### Plotting Data: #####
# Map Theme:
plottheme <- ggplot() +
  theme_void() +
  # Mapping theme:
  theme(axis.title = element_blank(),
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        panel.border = element_blank(),
        plot.margin = margin(t = 0, 
                             r = 0, 
                             b = 0, 
                             l = 0, 
                             unit = "cm"),
        plot.title = element_text(hjust = 0.5),
        plot.background = element_rect(fill = "white", 
                                       color = "black",
                                       linewidth = 0),
        panel.background = element_rect(fill = "white", 
                                        color = "black",
                                        linewidth = 0),
        panel.grid.major.x = element_line(color = "lightgrey",
                                          linetype = 2,
                                          linewidth = 0),
        panel.grid.minor.x = element_line(color = "lightgrey",
                                          linetype = 2,
                                          linewidth = 0),
        panel.grid.major.y = element_line(color = "grey",
                                          linetype = 2,
                                          linewidth = 0),
        panel.grid.minor.y = element_line(color = "grey",
                                          linetype = 2,
                                          linewidth = 0),
        axis.line.x.top = element_line(color = "white",
                                       linetype = 2,
                                       linewidth = 0),
        axis.line.y.right = element_line(color = "white",
                                         linetype = 2,
                                         linewidth = 0),
        axis.line.x.bottom = element_line(color = "black",
                                          linetype = 1,
                                          linewidth = 0),
        axis.line.y.left = element_line(color = "black",
                                        linetype = 1,
                                        linewidth = 0),
        # Text formatting:
        text = element_text(family = "serif", # font
                            size = 12, # font size
                            colour = "black"# font color
        ),
        legend.position = c(0.95, -0.05),
        legend.key = element_rect(color = "black", 
                                  fill = NA, 
                                  linewidth = 0.05, 
                                  linetype = 1),
        legend.justification = "right",
        legend.direction = "horizontal")

: 

# 2. Import data

Import necessary data.

## 2.1 Tomato

-   Yield = Total tomato production (total bucket of 25 lb) from 1 acres
    of land which varies from 10% to 200% of total production (100%).
    The range was simulated by multiplying 100% yield by yldvar.

-   yldvar = Yield variation parameter ranges from 10% to 200%.

-   Rev17 to Rev23 = Revenue for price ranges of \$17 to \$23 per bucket
    of tomato.

-   Total cost = Total cost of production for the given yield.

-   rolac17 to rolac23= Return to operator, labor and capital for price
    range of \$17 to \$23.

-   operator Cost = Operator labor cost at \$15/hour for given yield.
    For 100% yield, total hours = 90.

-   rlc17 to 23 = Return to land and capital after subtracting operator
    cost from total revenue.

In [ ]:
tomato <- read_xlsx("Parameters.xlsx",
                    sheet = "Tomato",
                    start_row = 2,
                    start_col = 9,
                    skip_empty_rows = TRUE,
                    skip_empty_cols = TRUE,
                    col_names = TRUE) %>% 
  rename(yield = Yield,
         yldvar = `Yield Variation (%)`)
dim(tomato)

: 

## 2.2 Strawberry

-   Everything same as tomato.

-   Numbers 3 to 9 in names are price ranges for strawberry.

In [ ]:
strawberry <- read_xlsx("Parameters.xlsx",
                        sheet = "Strawberry",
                        start_row = 2,
                        start_col = 7,
                        skip_empty_rows = TRUE,
                        skip_empty_cols = TRUE,
                        col_names = TRUE) %>% 
  rename(yield = Yield,
         yldvar = `Yield Variation (%)`)
dim(strawberry)

: 

## 2.3 Squash

-   Everything same as tomato and strawberry.

-   Numbers 11 to 17 in names are price ranges for squash.

In [ ]:
squash <- read_xlsx("Parameters.xlsx",
                    sheet = "Squash",
                    start_row = 2,
                    start_col = 8,
                    skip_empty_rows = TRUE,
                    skip_empty_cols = TRUE,
                    col_names = TRUE) %>% 
  rename(yield = Yield,
         yldvar = `Yield Variation (%)`)
dim(squash)

: 

## 2.4 Electricity price

Electricity price ranges from 1 cents to 6 cents in 0.5 cent increment.
Previously, I used AL retail electricity price as described below. It’s
no longer in use but I description below here for the record.

Electricity price (\$/kWh) was retail electricity price range for
Alabama based on retail electricity price in April 2023 and April 2024
taken from [DOE
Database](https://www.eia.gov/electricity/monthly/epm_table_grapher.php?t=epmt_5_6_a).
Retail electricity price range in Alabama was from 6.44 to 15.85
cents/kWh in April 2023 and April 2024 which represents industry,
commercial, and residential prices.

In [ ]:
elec_price <- read_xlsx("Parameters.xlsx",
                              sheet = "Electricity Price") %>%
  rename(epr_kwh = `Electricity Price ($/kWh)`)
dim(elec_price)

: 

## 2.5 PV system cost

-   Data taken from
    “<a href="https://www.nrel.gov/docs/fy21osti/77811.pdf"
    class="uri">Capital Costs for Dual-Use Photovoltaic Installations: 2020
    Benchmark</a>” Table 1 and Figure 3.
-   This data was used to estimate CAPEX.
-   avtyps = agrivoltaic types.
-   item = itemized component of system.
-   cost = cost of each item.
-   height = ground to panel clearance height (ft.)
-   tcost = Total cost is the sum of all itemized cost for AV system.
    See figure 3 and table 1 in above document for more detail.

In [ ]:
pvsc <- wb_read(file = "Parameters.xlsx",
                sheet = "PV system Cost (NREL)",
                rows = c(1:109),
                cols = c(1:5),
                col_names = TRUE) %>%
  rename(avtyps = `AV Types`,
         item = Item,
         cost = `Cost ($/W)`,
         height = `Panel Height (ft.)`,
         tcost = `Total Cost ($/W)`)
dim(pvsc)

: 

## 2.6 Capex (NREL)

Variable Descriptions:

-   Capex: Capital investment cost (\$/W) to develop solar energy
    system. Capex includes cost of physical structure, developer’s
    overhead and EPC/Developer’s net profit.

-   capex estimated as f(height, tracker) using OLS for 6.4 ft Tracking
    system.

-   Height = ground to panel clearance in ft.

-   array: Solar array. Tracker = Single axis sun tracking panels; Fixed
    = Non-tracking panels.

In [ ]:
capex <- read.table(file = "CAPEX.txt",
                    header = TRUE,
                  sep = "\t") %>% 
  rename(capex = cost,
         height = pheight,
         array = tracker)
dim(capex)

: 

### 2.6.1 Plotting capex

In [ ]:
plottheme %>% ggplot(data = capex,
                     mapping = (aes(
                       x = height,
                       y = capex,
                       color = array,
                       group = array))) +
  geom_point() +
  geom_line() +
  # geom_text(aes(label = "Tracker"),
  #           nudge_x = 0.05,
  #           nudge_y = 0.05,
  #           size = 6) +
  labs(
    title = "CAPEX Cost by Solar Panel Height",
    x = "Panel Height (ft.)",
    y = "CAPEX Cost ($/W)",
    color = "Array"
    ) +
  scale_x_continuous(limits = c(4.5, 8.5)) +
  scale_y_continuous(limits = c(1.5, 2.5)) +
  guides(color = guide_legend(reverse = TRUE))

: 

## 2.7 Panel Configuration

-   Panel configuration and DV system output (W).

In [ ]:
panconf <- wb_read(file = "Parameters.xlsx",
                sheet = "Panel Spacing",
                start_row = 2,
                start_col = 1,
                skip_empty_rows = TRUE,
                skip_empty_cols = TRUE,
                col_names = TRUE)
  # rename(avtyps = `AV Types`,
  #        item = Item,
  #        cost = `Cost ($/W)`,
  #        height = `Panel Height (ft.)`,
  #        tcost = `Total Cost ($/W)`)
dim(panconf)

: 

## 2.8 Energy output

Energy output was simulated using NREL
<a href="https://pvwatts.nrel.gov/pvwatts.php" class="uri">PV Watts
Calculator</a>.

-   sprop = land proportion covered by solar in 1 acres. Value ranges
    from 0 to 1.

-   Panels = Total number of panels in 1 acres of land.

-   datalot: 1 = first simulation done for four regions of AL; 2 =
    second simulation done for four regions of AL. Two simulations have
    two unique zipcodes for each simulated region.

-   al_regs = regions of Alabama

-   zips = zipcodes selected from each region of AL for simulation.

-   array = Fixed (open rack); 1AxisRot = 1 Axis Tracking. See above
    NREL tool for more detail.

-   dc_kw = DC system size, calculated for each solar panel heights
    considering solar panels efficiency and area covered by solar
    panels.

-   energy = total energy output ( kWh/Year) considering system
    parameters. Total hours considered by the model is 8,760 (See
    <a href="https://pvwatts.nrel.gov/pvwatts.php" class="uri">PV Watts
    Calculator</a> Results \> help (below the result) \> results \>
    download monthly or hourly results).

In [ ]:
energy_output <- read_xlsx("Parameters.xlsx", 
                           sheet = "Energy Output",
                           start_row = 1,
                           start_col = 1,
                           skip_empty_rows = TRUE,
                           skip_empty_cols = TRUE,
                           col_names = TRUE) %>%
  rename(sprop = `Solar Proportion`,
         panels = `Total Panels`,
         datalot = DataLot,
         al_regs = `Region of AL`,
         zips = ZIPCODE,
         array = `Array Type`,
         dc_kw = `DC System Size (kW)`,
         energy = `Energy (kWh/Year)`) %>%
  mutate(dc_kw = round(dc_kw, 2),
         array = case_when(
           array == "1AxisRot" ~ "Tracking",
           array == "FixedOpen" ~ "Fixed",
           TRUE ~ array))

dim(energy_output)

: 

### 2.8.1 Energy output by solar panels counts

Plotting Energy output by number of solar panels in one acres of AV
system from fixed and single axis rotation system for two zipcodes (1,
2) within each of the four regions of AL.

In [ ]:
lox <- c("Northern", "Central", "Black Belt", "Southern")
array_levs = c("Single Axis Rotation", "Fixed Open Rack")
datalot_levs = c("Location 1", "Location 2")
ggplot(data = energy_output,
         mapping = aes(x = al_regs,
                       y = energy,
                       #fill = energy,
                       color = factor(panels),
                       group = factor(panels))) +
  geom_line()+
  geom_point() +
  facet_grid(datalot~array) +
  scale_x_discrete(limits = lox,
                   labels = c("North", "Center", "B Belt", "South")) +
  guides(color = guide_legend(ncol = 2, reverse = TRUE))

: 

### 2.8.2 Energy output by DC System Size

Plotting Energy output by DC System Size from fixed and single axis
rotation system for two zipcodes (1, 2) within each of the four regions
of AL.

In [ ]:
lox <- c("Northern", "Central", "Black Belt", "Southern")
ggplot(data = energy_output,
         mapping = aes(x = al_regs,
                       y = energy,
                       #fill = energy,
                       color = factor(dc_kw),
                       group = factor(dc_kw))) +
  geom_line()+
  geom_point() +
  facet_grid(datalot~array) +
  scale_x_discrete(limits = lox,
                   labels = c("North", "Center", "B Belt", "South")) +
  guides(color = guide_legend(ncol =2, reverse = TRUE))

: 

# 3. Revenue from energy

## 3.1 Simulation 1

-   elcprc = electricity price. See Electricity price data for more
    detail.

-   elcrev = Revenue from electricity for given electricity prices. See
    “energy output” and “electricity price” dataset for more details.

-   I took average of “energy” from datalot 1 and datalot 2 to minimize
    computation time.

In [ ]:
# Convert to data frames if they are not already
matrix1 <- energy_output  %>%
  group_by(sprop, al_regs, array, dc_kw, panels) %>%
  #filter(datalot == 2) %>%
  # Compute mean of datalot 1 and datalot 2:
  summarise(
    energy = mean(energy),
    .groups = 'drop'
    ) # dimension of matrix is 168*6
matrix2 <- elec_price # dimension of matrix is 11*1

# Initialize the result data frame
# energy_revenue <- data.frame(matrix(nrow = 1848, ncol = 9))
energy_revenue <- data.frame(
  matrix(nrow = nrow(matrix2)*nrow(matrix1),
         ncol = ncol(matrix2)+ncol(matrix1)+1))

# Variable to keep track of the row index in the result matrix
row_index <- 1

# Loop through each value of the second matrix
for (i in 1:nrow(matrix2)) {
  # Loop through each value of the second matrix
  for (j in 1:nrow(matrix1)) {
    # First matrix, second matrix, combined two matrices.
    new_row <- c(matrix1[j, ], 
                 matrix2[i, ], 
                 matrix1$energy[j] * matrix2$epr_kwh[i])
    # Assign the new row to the result matrix
    energy_revenue[row_index, ] <- new_row
    # Increment the row index
    row_index <- row_index + 1
  }
}
# Name the columns
colnames(energy_revenue) <- c(colnames(matrix1), "elcprc", "elcrev")

# Display the result
dim(energy_revenue)

: 

## 3.2 Simulation 2

This simulation has same result as above (Cross checking above code and
output). Results are suppressed but errors and warnings are not. No
error and no warnings means code is working as it should.

In [ ]:
## | results='hide'
# Sample data
set.seed(123)
matrix1 <- energy_output # dimension of matrix is 176*7
matrix2 <- elec_price # dimension of matrix is 11*1

# Initializing the result matrix
result_matrix <- data.frame(matrix(ncol = nrow(matrix2),
                                   nrow = 0))
colnames(result_matrix) <- c(colnames(matrix1),  "elcrev", "elcprc")

# Loop to multiply first and second matrices
for (i in 1:nrow(matrix2)) {
  temp_matrix <- matrix1
  temp_matrix$E_Prc <- matrix2[i, ]
  temp_matrix$E_Rev <- matrix1$energy[j] * matrix2$epr_kwh[i]
  result_matrix <- rbind(result_matrix, temp_matrix)
}

# Display the resulting matrix
dim(result_matrix)
head(result_matrix)
tail(result_matrix)

: 

## 3.3 Plotting revenue from energy production

### 3.3.1 Breakdown by number of solar panels

I am using data from simulation 1 for this visualization. This code
plots one chart per electricity cost. There are 11 electricity cost
resulting into 11 charts.

In [ ]:
lox <- c("Northern", "Central", "Black Belt", "Southern")
array_levs = c("Single Axis Rotation", "Fixed Open Rack")
datalot_levs = c("Location 1", "Location 2")
for (i in unique(energy_revenue$elcprc)) {
 a = ggplot(data = (energy_revenue %>%
  filter(elcprc == i)),
         mapping = aes(x =al_regs,
                       y = elcrev,
                       #fill = energy,
                       color = factor(panels),
                       group = factor(panels)))+
  geom_line()+
  geom_point()+
  facet_grid(.~array) +
  scale_x_discrete(limits = lox,
                   labels = c("North", "Center", "B Belt", "South")) +
   guides(color = guide_legend(ncol =2, reverse = TRUE))
 cat("Electricity Price = ", i)
 print(a)
}

: 

### 3.3.2 Breakdown by proportion of land under solar panels

-   Two proportions may have same number of solar panels (Eg. 0.80 and
    0.85, 0.20 and 0.25). So, total lines in the chart may not match
    with total number of legend levels. Some proportions are overlapping
    in the chart. See panel configuration for more detail.

In [ ]:
lox <- c("Northern", "Central", "Black Belt", "Southern")
array_levs = c("Single Axis Rotation", "Fixed Open Rack")
datalot_levs = c("Location 1", "Location 2")
for (i in unique(energy_revenue$elcprc)) {
 a = ggplot(data = (energy_revenue %>%
  filter(elcprc == i)),
         mapping = aes(x =al_regs,
                       y = elcrev,
                       #fill = energy,
                       color = factor(sprop),
                       group = factor(sprop)))+
  geom_line()+
  geom_point()+
  facet_grid(.~array) +
  scale_x_discrete(limits = lox,
                   labels = c("North", "Center", "B Belt", "South")) +
   guides(color = guide_legend(ncol = 2, reverse = TRUE))
 cat("Electricity Price = ", i)
 print(a)
}

: 

# 4. Solar system cost

-   Cost of solar energy system in agrivoltaic setting.

-   I used DC system size (dc_kw) and capex (\$/W) to get total cost for
    each height and panel tracking system.

-   1 kw dc system size costs x\$ CAPEX for given height and tracking
    system.

-   y kw dc system size costs \$x\*y for given height and tracking
    system.

-   Use simulation 1 data and capex data to get solar system cost.

-   There should be 1936\*3 = 5808 rows in this dataset.

-   height = height of solar panels; see capex dataset for details.

-   capex = capex from capex table; see capex dataset for details.

-   ttlcost = Total cost for given DC system size.

-   anncost = Annual payment to repay loan ($P_{ann}$) =
    $\frac{P_o(i(1 + i)^t)}{(1+i)^t - 1)}$ , where $P_o$ = CAPEX loan
    burrowed to repay in $t$ years; $t = 25$, and $i$ = annual interest
    rate at $5\%$.

-   moncost = Monthly payment to repay loan ($P_{mon}$) =
    $\frac{P_o((i/12)(1 + (i/12))^{t*12})}{(1+(i/12))^{t*12} - 1)}$,
    where $P_o$ = CAPEX loan burrowed to repay in $t$ years; $t = 25$,
    and $i$ = annual interest rate at $5\%$.

In [ ]:
expanded_data <- energy_revenue %>%
  slice(rep(1:n(),
            each = 3))
capex_height <- rep(unique(capex$height),
                    length.out = nrow(energy_revenue))
energy_cost = cbind(expanded_data, capex_height) %>% 
  rename(height = capex_height)

energy_cost <- left_join(energy_cost, 
                         capex, 
                         by = c("array", "height")) %>% 
  mutate(ttlcost = capex*dc_kw,
         anncost = ttlcost*(0.05*(1 + 0.05)^25)/
           ((1 + 0.05)^25 - 1),
         moncost = ttlcost*((0.05/12)*(1 + (0.05/12))^(25*12))/
           ((1 + (0.05/12))^(25*12) - 1))

dim(energy_cost)

: 

# 5. Profit from solar

Profit from solar energy system in agrivoltaic setting

-   eprofit = profit from electricity after subtracting total cost
    (ttlcost) from total revenue (elcrev).
-   eannprof = annual profit from solar after subtracting annual loan
    repayment distributed over 25 years.
-   emonprof = monthly profit from solar after subtracting monthly loan
    repayment distributed over 25 years.

In [ ]:
solar_profit <- energy_cost %>%
  mutate(eprofit = elcrev - ttlcost,
         eannprof = elcrev - anncost,
         emonprof = (elcrev/12) - moncost)
dim(solar_profit)

: 

## 5.1 Plot profit from solar

In [ ]:
lox <- c("Northern", "Central", "Black Belt", "Southern")
array_levs = c("Single Axis Rotation", "Fixed Open Rack")
datalot_levs = c("Location 1", "Location 2")
  for (i in unique(solar_profit$elcprc)) {
    b = ggplot(
      data = (solar_profit %>%
                filter(elcprc == i)),
      mapping = aes(
        x = al_regs,
        y = eprofit,
        #fill = energy,
        color = factor(panels),
        group = factor(panels)
      )
    ) +
      geom_line() +
      geom_point() +
      facet_grid(height ~ array) +
      scale_x_discrete(limits = lox,
                       labels = c("North", "Center", 
                                  "B Belt", "South")) +
      guides(color = guide_legend(ncol = 2, 
                                  reverse = TRUE))
    cat("Electricity Price = ", i)
    print(b)
  }

: 

# 6. Profit from crops

## 6.1 Tomato

Filter return to operator, land and capital profit from Tomato:

In [ ]:
tomato_profit = tomato %>% 
  select(yldvar, yield, 
         rolac17, rolac18, rolac19, rolac20, 
         rolac21, rolac22, rolac23)
dim(tomato_profit)

: 

Convert data to long format:

In [ ]:
# Assign column names for clarity
colnames(tomato_profit) <- c("yldvar", "yield",
                  "rolac17", "rolac18", "rolac19",
                  "rolac20", "rolac21", "rolac22",
                  "rolac23")

# Reshape the data frame from wide to long format
tomato_long <- melt(tomato_profit,
                id.vars = c("yldvar", "yield"),
                measure.vars = c("rolac17", "rolac18", "rolac19",
                                 "rolac20", "rolac21", "rolac22",
                                 "rolac23"),
                variable.name = "price",
                value.name = "profit")

# Convert the 'Price' column to numeric by extracting the number
tomato_long$price <- as.numeric(gsub("rolac", "", tomato_long$price))

# View the resulting data frame
dim(tomato_long)

: 

### 6.1.1 Profit from tomato

In [ ]:
ggplot(data = tomato_long,
       mapping = aes(x = price,
                     y = profit,
                     color = factor(yldvar),
                     group = factor(yield))) +
  geom_line() +
  geom_point() +
  geom_hline(yintercept = 0,
             linetype = "dashed",
             color = "black") +
  guides(color = guide_legend(ncol = 2,
                              reverse = TRUE))

: 

In [ ]:
ggplot(data = tomato_long,
       mapping = aes(x = yield,
                     y = profit,
                     #fill = yield,
                     color = factor(price),
                     group = factor(price))) +
  geom_line() +
  geom_point() +
  geom_hline(yintercept = 0,
             linetype = "dashed",
             color = "black") +
  # Vertical dashed line is 100% yield
  geom_vline(xintercept = tomato_long$yield[11],
             linetype = "dashed",
             color = "black") +
guides(color = guide_legend(reverse = TRUE))

: 

## 6.2 Strawberry

Filter return to operator, land and capital profit from strawberry

In [ ]:
strawberry_profit = strawberry %>% 
  select(yldvar, yield, 
         rolac3, rolac4, rolac5, rolac6, 
         rolac7, rolac8, rolac9)
dim(strawberry_profit)

: 

Convert data to long format:

In [ ]:
# Assign column names for clarity
colnames(strawberry_profit) <- c("yldvar", "yield",
                  "rolac3", "rolac4", "rolac5",
                  "rolac6", "rolac7", "rolac8",
                  "rolac9")
# Reshape the data frame from wide to long format
stberry_long <- melt(strawberry_profit,
                id.vars = c("yldvar", "yield"),
                measure.vars = c("rolac3", "rolac4", "rolac5",
                                 "rolac6", "rolac7", "rolac8",
                                 "rolac9"),
                variable.name = "price",
                value.name = "profit")

# Convert the 'Price' column to numeric by extracting the number
stberry_long$price <- as.numeric(gsub("rolac", "", stberry_long$price))

# View the resulting data frame
dim(stberry_long)

: 

### 6.2.1 Plot Strawberry Profit

In [ ]:
ggplot(data = stberry_long,
       mapping = aes(x = price,
                     y = profit,
                     color = factor(yldvar),
                     group = factor(yield))) +
  geom_line() +
  geom_point() +
  geom_hline(yintercept = 0,
             linetype = "dashed",
             color = "black") +
  guides(color = guide_legend(ncol = 2,
                              reverse = TRUE))

: 

In [ ]:
ggplot(data = stberry_long,
       mapping = aes(x = yield,
                     y = profit,
                     color = factor(price),
                     group = factor(price))) +
  geom_line() +
  geom_point() +
  geom_hline(yintercept = 0,
             linetype = "dashed",
             color = "black") +
  #Vertical dashed line is 100% yield
  geom_vline(xintercept = stberry_long$yield[11],
             linetype = "dashed",
             color = "black") +
  guides(color = guide_legend(reverse = TRUE))

: 

## 6.3 Squash

Filter return to operator, land and capital profit from squash

In [ ]:
squash_profit = squash %>% 
  select(yldvar, yield, 
         rolac11, rolac12, rolac13, rolac14, 
         rolac15, rolac16, rolac17)
squash_profit

: 

Convert data to long format:

In [ ]:
# Assign column names for clarity
colnames(squash_profit) <- c("yldvar", "yield",
                  "rolac11", "rolac12", "rolac13",
                  "rolac14", "rolac15", "rolac16",
                  "rolac17")

# Reshape the data frame from wide to long format
squash_long <- melt(squash_profit,
                id.vars = c("yldvar", "yield"),
                measure.vars = c("rolac11", "rolac12", "rolac13",
                                 "rolac14", "rolac15", "rolac16",
                                 "rolac17"),
                variable.name = "price",
                value.name = "profit")

# Convert the 'Price' column to numeric by extracting the number
squash_long$price <- as.numeric(gsub("rolac", "", squash_long$price))

# View the resulting data frame
dim(squash_long)

: 

### 6.3.1 Profit from squash:

In [ ]:
ggplot(data = squash_long,
       mapping = aes(x = price,
                     y = profit,
                     color = factor(yldvar),
                     group = factor(yield))) +
  geom_line() +
  geom_point() +
  geom_hline(yintercept = 0,
             linetype = "dashed",
             color = "black") +
  guides(color = guide_legend(ncol = 2, 
                              reverse = TRUE))

: 

In [ ]:
ggplot(data = squash_long,
       mapping = aes(x = yield,
                     y = profit,
                     color = factor(price),
                     group = factor(price))) +
  geom_line() +
  geom_point() +
  geom_hline(yintercept = 0,
             linetype = "dashed",
             color = "black") +
  # Vertical dashed line is 100% yield
    geom_vline(xintercept = squash_long$yield[11],
             linetype = "dashed",
             color = "black") +
  guides(color = guide_legend(reverse = TRUE))

: 

# 7. Profit from agrivoltaics

Total profit from solar and crops for all combinations of AVs simulated.

## 7.1 Profit from tomato agrivoltaic system

-   Joint profit from tomato (tomato_long) and solar energy production
    (solar_profit) from 1 acre of land.
-   The last variable (tav_profit) is the final profit from tomato
    agrivoltaic system which is the result of our interest.

In [ ]:
# Generate all combinations of row indices from both matrices
index_combinations <- expand.grid(1:nrow(solar_profit),
                                  1:nrow(tomato_long))

# Define a function to process each combination of indices
process_combination <- function(indices) {
  i <- indices[1]
  j <- indices[2]
  new_row <- c(solar_profit[i, ],
               tomato_long[j, ], 
               #solar_profit[i, 14] = eannprof
               solar_profit$eannprof[i] + tomato_long$profit[j]) 
  return(new_row)
}

# Apply the function to each combination of indices and combine the results into a matrix
tav_profit <- do.call(rbind,
                          lapply(
                            seq_len(nrow(index_combinations)),
                            function(k) {
                              indices <- as.integer(
                                index_combinations[k, ])
                              process_combination(indices)
                              }))

# Optionally, you can convert the result back to a data frame if needed
tav_profit <- as.data.frame(tav_profit) %>%
    rename(tav_profit = V21)
tav_profit <- data.frame(lapply(tav_profit, unlist))
str(tav_profit)

: 

### 7.1.1 Saving results locally

In [ ]:
#write_csv(tav_profit, "tav_profit.csv")
write_feather(tav_profit,
  sink = "tav_profit.feather",
  version = 2,
  chunk_size = 65536L,
  compression = c("default"),
  #compression = c("default", "lz4", "lz4_frame", "uncompressed", "zstd"),
  compression_level = NULL
)

: 

## 7.2 Profit from strawberry agrivoltaic system

-   Joint profit from strawberry (stberry_long) and solar energy
    production (solar_profit) from 1 acre of land.
-   The last variable (sbav_profit) is the final profit from strawberry
    agrivoltaic system which is the result of our interest.

In [ ]:
# Generate all combinations of row indices from both matrices
index_combinations <- expand.grid(1:nrow(solar_profit),
                                  1:nrow(stberry_long))

# Define a function to process each combination of indices
process_combination <- function(indices) {
  i <- indices[1]
  j <- indices[2]
  new_row <- c(solar_profit[i, ],
               stberry_long[j, ], 
               #solar_profit[i, 14] = eannprof
               solar_profit$eannprof[i] + stberry_long$profit[j])
  return(new_row)
}

# Apply the function to each combination of indices and combine the results into a matrix
sbav_profit <- do.call(rbind, 
                          lapply(
                            seq_len(nrow(index_combinations)),
                            function(k) {
                              indices <- as.integer(
                                index_combinations[k, ])
                              process_combination(indices)
                              }))

# Optionally, you can convert the result back to a data frame if needed
sbav_profit <- as.data.frame(sbav_profit) %>%
  rename(sbav_profit = V21)
sbav_profit <- data.frame(lapply(sbav_profit, unlist))
str(sbav_profit)

: 

### 7.2.1 Saving results locally

In [ ]:
#write_csv(sbav_profit, "tav_profit.csv")
write_feather(sbav_profit,
  sink = "sbav_profit.feather",
  version = 2,
  chunk_size = 65536L,
  compression = c("default"),
  #compression = c("default", "lz4", "lz4_frame", "uncompressed", "zstd"),
  compression_level = NULL
)

: 

## 7.3 Profit from squash agrivoltaic system

-   Joint profit from squash (squash_long) and solar energy production
    (solar_profit) from 1 acre of land.
-   The last variable (sqav_profit) is the final profit from squash
    agrivoltaic system which is the result of our interest.

In [ ]:
# Generate all combinations of row indices from both matrices
index_combinations <- expand.grid(1:nrow(solar_profit),
                                  1:nrow(squash_long))

# Define a function to process each combination of indices
process_combination <- function(indices) {
  i <- indices[1]
  j <- indices[2]
  new_row <- c(solar_profit[i, ],
               squash_long[j, ],
               #solar_profit[i, 14] = eannprof
               solar_profit$eannprof[i] + squash_long$profit[j])
  return(new_row)
}

# Apply the function to each combination of indices and combine the results into a matrix
sqav_profit <- do.call(rbind, 
                          lapply(
                            seq_len(nrow(index_combinations)),
                            function(k) {
                              indices <- as.integer(
                                index_combinations[k, ])
                              process_combination(indices)
                              }))

# Optionally, you can convert the result back to a data frame if needed
sqav_profit <- as.data.frame(sqav_profit) %>%
  rename(sqav_profit = V21)
sqav_profit <- data.frame(lapply(sqav_profit, unlist))
str(sqav_profit)

: 

### 7.3.1 Saving results locally

In [ ]:
#write_csv(sqav_profit, "tav_profit.csv")
write_feather(sqav_profit,
  sink = "sqav_profit.feather",
  version = 2,
  chunk_size = 65536L,
  compression = c("default"),
  #compression = c("default", "lz4", "lz4_frame", "uncompressed", "zstd"),
  compression_level = NULL
)

: 